Il faut :
- prendre nos datas
- créer les emmbeddings avec la partie texte (extraire)
- faire une fonction pour trouver les similarités - faire ressortir les 3 documents les plus similaires par exemple
- fine tuner ensuite sur nos données- https://huggingface.co/blog/how-to-train-sentence-transformers
https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
https://www.codegram.com/blog/finding-similar-documents-with-transformers/
https://huggingface.co/facebook/bart-large-mnli
https://huggingface.co/docs/transformers/tasks/question_answering

% pip install -U sentence-transformers

In [1]:
from typing import List
from sentence_transformers import SentenceTransformer

## Extraire le contenu du csv

In [7]:
import csv
with open("JADe_project1/Data/movie_synopsis.csv") as f:
    content = []
    csv_reader = csv.reader(f)
    for line in csv_reader:
        # process each line
        if line[0] == "":
            print(line)
        else:
            content.append(line)

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## User Input

In [9]:
input_ju = "A love story between a human and a non-human."
input_jus = model.encode(input_ju)

## Télécharger le modèle et créer les vecteurs

In [11]:
from sentence_transformers import SentenceTransformer
sentences = []
for id,synopsis,title in content:
    # on récupère seulement le synopsis
    sentences.append(synopsis)

# chargement du modèle
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

## Créer un objet contenant les vecteurs et les titres

In [12]:
from collections import namedtuple

# Définir un named tuple pour représenter des coordonnées géographiques
Vecteurs = namedtuple('Vecteurs', ['embedding', 'titre'])

# Créer une instance de named tuple
total = []
for em,(id,synopsis,title) in zip(embeddings,content):
    total.append(Vecteurs(embedding=em, titre=title))

#print(total[1])

## Fonction qui donne les suggestion
Par défaut, on donne 5 suggestions. 

In [13]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [14]:
# Test
give_suggestion("A love story between a human and a non human.")

Suggestion 1: Bad Biology - distance [[0.8491142]]
Suggestion 2: Dedication - distance [[0.8841987]]
Suggestion 3: The Notebook - distance [[0.89577276]]
Suggestion 4: Youth Without Youth - distance [[0.92052394]]
Suggestion 5: Underground - distance [[0.9421078]]


[(array([[1.1355754]], dtype=float32), 'title'),
 (array([[1.2892183]], dtype=float32), 'Four Rooms'),
 (array([[1.354372]], dtype=float32), 'Judgment Night'),
 (array([[1.3075893]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[1.3326209]], dtype=float32), 'Star Wars'),
 (array([[1.3228948]], dtype=float32), 'Finding Nemo'),
 (array([[1.0564014]], dtype=float32), 'Forrest Gump'),
 (array([[1.2773778]], dtype=float32), 'American Beauty'),
 (array([[1.2825289]], dtype=float32), 'Citizen Kane'),
 (array([[1.1615679]], dtype=float32), 'Dancer in the Dark'),
 (array([[1.2811139]], dtype=float32), 'The Dark'),
 (array([[1.3072717]], dtype=float32), 'The Fifth Element'),
 (array([[1.3139929]], dtype=float32), 'My Life Without Me'),
 (array([[1.3734022]], dtype=float32), 'The Endless Summer'),
 (array([[1.2710116]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[1.3248636]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.2679561]], d

# Et avec les KNN ?

In [50]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier



def train_model(donnee:tuple):
    # jeux de données
    X = []
    for emb,_ in total:
        X.append(emb)
    y = [title for _,title in total]
    # créer une instance de LabelEncoder
    label_encoder = LabelEncoder()
    # convertir les labels en entiers
    label_encoder = LabelEncoder()
    label_encoder.fit(y)
    y_encoded = label_encoder.transform(y)

    # entraîner le classificateur KNN avec k=5 (5 voisins)
    # créer un classificateur KNN avec k=5 voisins
    knn = KNeighborsClassifier(n_neighbors=1)

    model = knn.fit(X,y_encoded)
    return model_knn

def give_suggestion_knn(user_input:str, model_knn, k:int=5):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    predicted_class_encoded = model_knn.predict([embed_user])
    # trouver les 3 voisins les plus proches du point d'entrée
    distances, indices = model_knn.kneighbors([embed_user], n_neighbors=k)
    # extraire les classes des 3 voisins les plus proches
    closest_classes = [y[i] for i in indices[0]]
    return closest_classes

In [51]:
give_suggestion_knn("Deux chats sont amis, ils leur arrivent plein de choses.")

['Magique !',
 'The one I love',
 'Une  approche positive du comportement des perroquets',
 'Uffe Holms Flash back',
 'Célibataires']

## Fine-tuning triplet loss

Faire un corpus de 50 samples - format : 
- Query 
- Synopsis
- Title 

Et utiliser triplet loss pour fine-tuned le modèle 

In [16]:
import csv
with open("Data/movie_synopsis.csv") as f:
    content = []
    csv_reader = csv.reader(f)
    for line in csv_reader:
        # process each line
        if line[0] == "":
            print(line)
        else:
            content.append(line)

In [18]:
from collections import namedtuple
import random

# Définir un named tuple pour représenter des coordonnées géographiques
Query_FT = namedtuple('Query_FT', ['anchor_text', 'positive_text','negative_text'])

# Créer une instance de named tuple
liste_for_fine_tuned = []
for id, syn, title, q in content[1:]:
    if q != "":
        neg = random.choice(content[:32])[3]
        if neg != syn:
            liste_for_fine_tuned.append(Query_FT(anchor_text=syn, positive_text=q, negative_text=neg))
        else : 
            neg = random.choice(content[:32])[3]
            liste_for_fine_tuned.append(Query_FT(anchor_text=syn, positive_text=q, negative_text=neg))
            

In [19]:
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
train_examples = [InputExample(texts=[anchor_text, positive_text, negative_text]) for anchor_text, positive_text, negative_text in liste_for_fine_tuned]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True) #, batch_size=32
train_loss = losses.TripletLoss(model=model)

In [20]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

Iteration:   0%|          | 0/49 [00:00<?, ?it/s]

In [21]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [34]:
give_suggestion("love story that happend in space")

Suggestion 1: Youth Without Youth - distance [[0.9539802]]
Suggestion 2: The Notebook - distance [[0.97290546]]
Suggestion 3: Bad Biology - distance [[0.9935017]]
Suggestion 4: Meat Love - distance [[1.0142233]]
Suggestion 5: Angel Eyes - distance [[1.0230942]]


[(array([[1.1885422]], dtype=float32), 'title'),
 (array([[1.2404019]], dtype=float32), 'Four Rooms'),
 (array([[1.3257325]], dtype=float32), 'Judgment Night'),
 (array([[1.2574844]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[1.3107374]], dtype=float32), 'Star Wars'),
 (array([[1.349925]], dtype=float32), 'Finding Nemo'),
 (array([[1.2047448]], dtype=float32), 'Forrest Gump'),
 (array([[1.2494332]], dtype=float32), 'American Beauty'),
 (array([[1.3123653]], dtype=float32), 'Citizen Kane'),
 (array([[1.1744156]], dtype=float32), 'Dancer in the Dark'),
 (array([[1.2425354]], dtype=float32), 'The Dark'),
 (array([[1.2854378]], dtype=float32), 'The Fifth Element'),
 (array([[1.3702557]], dtype=float32), 'My Life Without Me'),
 (array([[1.2907767]], dtype=float32), 'The Endless Summer'),
 (array([[1.3232927]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[1.3287982]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.2157075]], d

In [25]:
model.save("model_V1")

In [28]:
fine_tuned_model = SentenceTransformer('model_V1')

In [47]:
fine_tuned_model.encode("A love story movie")

array([-6.16937317e-02,  3.39214951e-02,  1.63940582e-02,  8.23609903e-02,
       -4.23810743e-02,  6.67449385e-02,  1.03911795e-02, -1.20512219e-02,
        1.77744746e-01, -7.49209076e-02,  2.77365604e-03,  2.35413481e-02,
        3.02357040e-02,  6.26352131e-02,  2.52910275e-02,  2.74742246e-02,
       -2.28060950e-02, -1.75077356e-02, -5.60949333e-02, -2.13142429e-02,
        1.34124216e-02, -8.31889547e-03, -1.02875112e-02,  3.45553085e-03,
       -1.19861707e-01, -1.84850637e-02,  5.44298589e-02, -1.19243888e-02,
       -1.22677572e-01,  6.52009342e-03,  7.14267418e-02,  2.83184946e-02,
        5.87457232e-03,  2.43220553e-02, -4.44959365e-02, -1.14945397e-02,
       -5.19665554e-02,  1.79308541e-02,  6.83711004e-03, -9.95507278e-03,
       -1.59119274e-02, -1.26442149e-01,  9.13602635e-02,  3.41743901e-02,
       -3.19716497e-03, -3.51224355e-02,  9.91018210e-03, -2.08692364e-02,
        2.80503295e-02,  1.44855864e-02, -8.27746764e-02,  3.54465991e-02,
       -8.06295350e-02, -

In [29]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = fine_tuned_model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [31]:
give_suggestion("give me a story about a love story of dancer")

Suggestion 1: Melissa - distance [[0.94217503]]
Suggestion 2: Take the Lead - distance [[0.94227797]]
Suggestion 3: Breakin' - distance [[0.9480594]]
Suggestion 4: Make It Happen - distance [[0.9844067]]
Suggestion 5: I Capture the Castle - distance [[1.0032454]]


[(array([[1.1949]], dtype=float32), 'title'),
 (array([[1.2577446]], dtype=float32), 'Four Rooms'),
 (array([[1.373899]], dtype=float32), 'Judgment Night'),
 (array([[1.3842568]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[1.3765539]], dtype=float32), 'Star Wars'),
 (array([[1.3993158]], dtype=float32), 'Finding Nemo'),
 (array([[1.1965058]], dtype=float32), 'Forrest Gump'),
 (array([[1.305817]], dtype=float32), 'American Beauty'),
 (array([[1.3486058]], dtype=float32), 'Citizen Kane'),
 (array([[1.1647427]], dtype=float32), 'Dancer in the Dark'),
 (array([[1.2330817]], dtype=float32), 'The Dark'),
 (array([[1.3346256]], dtype=float32), 'The Fifth Element'),
 (array([[1.3178713]], dtype=float32), 'My Life Without Me'),
 (array([[1.354611]], dtype=float32), 'The Endless Summer'),
 (array([[1.2722754]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[1.310004]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.3811045]], dtype=f

## Crowd sourcing fine-tuning

In [43]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Define your train examples. You need more than just two examples...
# C'est ici qu'on peut utiliser les pouces ! On peut dire qu'un pouce positif c'est 0.8 et negatif c'est 0 par exemple.
# Et du coup on peut fine-tuner avec les avis des utilisateurs.
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
import csv
with open("JADe_project1/Data/movie_synopsis.csv") as f:
    content_bis = []
    csv_reader = csv.reader(f)
    for line in csv_reader:
        # process each line
        if line[0] == "":
            print(line)
        else:
            content_bis.append(line)

In [50]:
from sentence_transformers import SentenceTransformer
sentences = []
for id,synopsis,title in content_bis:
    # on récupère seulement le synopsis
    sentences.append(synopsis)

# chargement du modèle
fine_tuned_model = SentenceTransformer('model_V1')
embeddings = fine_tuned_model.encode(sentences)

In [51]:
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
def give_suggestion_finetuned(user_input:str,k:int=5):
    # on encode l'input de l'user avec le même modèle pour avoir un vecteur
    embed_user = fine_tuned_model.encode(user_input)
    stock_similarity = []
    stock_suggest_k = []
    # on calcule les distances cosinus
    for embed,title in total:
        #result = (cosine_distances([embed],[embed_user]),title)
        result = (euclidean_distances([embed],[embed_user]),title)
        # elles sont stockés dans la liste sous forme de tuple contenant la distance cosinus et le titre
        stock_similarity.append(result)
    # on trie (ordre croissant) pour avoir le synopsis le plus proche de l'input de l'usr   
    ma_liste_triee = sorted(stock_similarity, key=lambda x: x[0])
    # afficher les premiers résultats etc - on peut changer cette partie en fonction de ce qu'on veut afficher 
    #(idem pour le return)
    for i, (x, y) in enumerate(ma_liste_triee[:5]):
        print(f"Suggestion {i+1}: {y} - distance {x}")
        stock_suggest_k.append(y)
    return stock_similarity #ou stock_suggest_k ou stock_similarity[:k]

In [53]:
give_suggestion_finetuned("A love story between a human and a non human.")
Suggestion 1: Bad Biology - distance [[0.8491142]]
Suggestion 2: Dedication - distance [[0.8841987]]
Suggestion 3: The Notebook - distance [[0.89577276]]
Suggestion 4: Youth Without Youth - distance [[0.92052394]]
Suggestion 5: Underground - distance [[0.9421078]]

Suggestion 1: Bad Biology - distance [[0.84641194]]
Suggestion 2: Dedication - distance [[0.8905853]]
Suggestion 3: The Notebook - distance [[0.9077956]]
Suggestion 4: Youth Without Youth - distance [[0.92056304]]
Suggestion 5: Underground - distance [[0.950112]]


[(array([[1.1562754]], dtype=float32), 'title'),
 (array([[1.2979056]], dtype=float32), 'Four Rooms'),
 (array([[1.3588073]], dtype=float32), 'Judgment Night'),
 (array([[1.3174201]], dtype=float32), 'Life in Loops (A Megacities RMX)'),
 (array([[1.3318678]], dtype=float32), 'Star Wars'),
 (array([[1.3202047]], dtype=float32), 'Finding Nemo'),
 (array([[1.0482637]], dtype=float32), 'Forrest Gump'),
 (array([[1.273936]], dtype=float32), 'American Beauty'),
 (array([[1.2852248]], dtype=float32), 'Citizen Kane'),
 (array([[1.1621729]], dtype=float32), 'Dancer in the Dark'),
 (array([[1.2779489]], dtype=float32), 'The Dark'),
 (array([[1.308762]], dtype=float32), 'The Fifth Element'),
 (array([[1.3185875]], dtype=float32), 'My Life Without Me'),
 (array([[1.3815882]], dtype=float32), 'The Endless Summer'),
 (array([[1.2733253]], dtype=float32),
  'Pirates of the Caribbean: The Curse of the Black Pearl'),
 (array([[1.3344971]], dtype=float32), 'Kill Bill: Vol. 1'),
 (array([[1.2800916]], dt

### 